In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
import sys
sys.path.append('../')

In [3]:
import pandas as pd
import numpy as np
import time
from implicit.bpr import BayesianPersonalizedRanking
from scipy.sparse import coo_matrix

from src.preprocess import add_time_idx
from sklearn.model_selection import train_test_split
import optuna


libgomp: Invalid value for environment variable OMP_NUM_THREADS

libgomp: Invalid value for environment variable OMP_NUM_THREADS


In [5]:
DATA_PATH = '../data/'

In [6]:
datasets=["beer_advocate","gowalla","ml-20m","steam","twitch","yelp"]

In [9]:
from recommenders.evaluation.python_evaluation import map_at_k, ndcg_at_k, recall_at_k

def compute_metrics(ground_truth, preds, k=10):

    if not hasattr(ground_truth, 'rating'):
        ground_truth = ground_truth.assign(rating=1)

    # when we have 1 true positive, HitRate == Recall and MRR == MAP
    metrics = {
        'ndcg': ndcg_at_k(ground_truth, preds, col_user='user_id', col_item='item_id',
                          col_prediction='prediction', col_rating='rating', k=k),
        'recall': recall_at_k(ground_truth, preds, col_user='user_id', col_item='item_id',
                                col_prediction='prediction', col_rating='rating', k=k),
        'map': map_at_k(ground_truth, preds, col_user='user_id', col_item='item_id',
                        col_prediction='prediction', col_rating='rating', k=k)
    }

    return metrics

In [7]:
def score_dataset(dataset_name):
    data = pd.read_csv(f"{DATA_PATH}{dataset_name}_v2.csv")
    data = add_time_idx(data)
    
    train = data[data.time_idx_reversed >= 10]
    test = data[data.time_idx_reversed < 10]

    users_validation, users_test = train_test_split(
        test.user_id.unique(), test_size=0.5, random_state=42)
    validation = test[test.user_id.isin(users_validation)]
    test = test[test.user_id.isin(users_test)]

    train = add_time_idx(train)
    validation = add_time_idx(validation)
    test = add_time_idx(test)
    
    test_matrix= coo_matrix((np.ones(len(train)), (train.user_id, train.item_id-1)), (data.user_id.max()+1, data.item_id.max()))
    
    study = optuna.create_study(direction='maximize')
    study.optimize(lambda trial: objective(trial, test_matrix,data,validation), n_trials=40)
    best_params = study.best_params
    best_value = score_one_attempt(test_matrix,data,test, onlyTrial= False,**best_params)
    
   
    
    with open(f"bpr_{dataset_name}.json","w") as f:
         f.write(f"""{{"time": {best_value[1]},
            "recall10": {best_value[2]},
            "ndcg10": {best_value[0]},
            "map10": {best_value[3]}}}\n""")
            
            
def objective(trial, test_matrix,data,test):
    factors = trial.suggest_int('factors', 3, 10)
    learning_rate = trial.suggest_int('learning_rate', -4, -2)
    regularization = trial.suggest_int('regularization', -3, 0)
  
    return score_one_attempt(test_matrix,data,test,factors,learning_rate,regularization)
    
def score_one_attempt(test_matrix,data,test,factors,learning_rate,regularization, onlyTrial=True):
    time_list=[]
    recall10 = []
    ndcg10 = []
    map10 = []
    
    for i in range(10):
        seed = np.random.randint(1000)
        bpr = BayesianPersonalizedRanking(use_gpu=True,random_state=seed,iterations=200,
                                         factors =int(2**factors),
                                         learning_rate=10**learning_rate,
                                         regularization=10**regularization)
        start_time = time.time()
        bpr.fit(test_matrix)
        time_list.append(time.time() - start_time)

        result = bpr.recommend(np.arange(data.user_id.max()+1),
                  test_matrix.tocsr(),
                  100,
                  True,
                  recalculate_user= False)
        result_df =pd.DataFrame(columns=["user_id","item_id","prediction"])
        result_df["user_id"]=np.repeat(np.arange(data.user_id.max()+1), 100)
        result_df["item_id"]=result[0].ravel()+1
        result_df["prediction"]=result[1].ravel()
        metrics = compute_metrics(test, result_df, k=10)
        ndcg10.append(metrics["ndcg"])
        recall10.append(metrics["recall"])
        map10.append(metrics["map"])

    return np.mean(ndcg10) if onlyTrial else (np.mean(ndcg10),np.mean(time_list),np.mean(recall10),np.mean(map10))
       
    

In [ ]:
for dataset in datasets:
    score_dataset(dataset)

[I 2023-10-12 19:53:06,687] A new study created in memory with name: no-name-21f3a162-5f99-40d5-a352-177897101124
/home/jovyan/.imgenv-gpu-8a100-0/lib/python3.7/site-packages/implicit/utils.py:167: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.06499409675598145 seconds
  ParameterWarning,


  0%|          | 0/200 [00:00<?, ?it/s]

/home/jovyan/.imgenv-gpu-8a100-0/lib/python3.7/site-packages/implicit/utils.py:167: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.06820559501647949 seconds
  ParameterWarning,


  0%|          | 0/200 [00:00<?, ?it/s]

/home/jovyan/.imgenv-gpu-8a100-0/lib/python3.7/site-packages/implicit/utils.py:167: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.06622457504272461 seconds
  ParameterWarning,


...

  0%|          | 0/200 [00:00<?, ?it/s]

/home/jovyan/.imgenv-gpu-8a100-0/lib/python3.7/site-packages/implicit/utils.py:167: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.05306506156921387 seconds
  ParameterWarning,


  0%|          | 0/200 [00:00<?, ?it/s]

/home/jovyan/.imgenv-gpu-8a100-0/lib/python3.7/site-packages/implicit/utils.py:167: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.052985191345214844 seconds
  ParameterWarning,


  0%|          | 0/200 [00:00<?, ?it/s]